<a href="https://colab.research.google.com/github/Trickshotblaster/nn-practices/blob/main/AIShakespeare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [2]:
import requests
url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
text = requests.get(url).text

In [35]:
vocab = sorted(set(text))
vocab_size = len(vocab) + 1
block_size = 5
stoi = {' ': 0}
itos = {0: ' '}
stoi.update({char:i+1 for i, char in enumerate(vocab)})
itos.update({i+1:char for i, char in enumerate(vocab)})
def build_dataset(text, block_size):
  xs = []
  ys = []
  block = [0] * block_size
  for char, next in zip(text, text[1:]):
    block = block[1:] + [stoi[char]]
    xs.append(block)
    ys.append(stoi[next])
  return xs, ys
def make_splits(text, block_size, split_amounts):
  train_text = text[0: int(len(text) * split_amounts[0])]
  dev_text = text[int(len(text) * split_amounts[0]): int(len(text) * split_amounts[0]) + int(len(text) * split_amounts[1])]
  test_text = text[int(len(text) * split_amounts[0]) + int(len(text) * split_amounts[1]):]
  Xtr, Ytr = build_dataset(train_text, block_size)
  Xdev, Ydev = build_dataset(dev_text, block_size)
  Xte, Yte = build_dataset(test_text, block_size)
  return torch.tensor(Xtr), torch.tensor(Ytr), torch.tensor(Xdev), torch.tensor(Ydev), torch.tensor(Xte), torch.tensor(Yte)

Xtr, Ytr, Xdev, Ydev, Xte, Yte = make_splits(text, block_size, [0.8, 0.1])
print(Xtr[0], Ytr[0])

tensor([ 0,  0,  0,  0, 19]) tensor(48)


In [57]:
emb_dim = 10
hidden_size = 100
C = torch.randn(vocab_size, emb_dim)
w1 = torch.randn(block_size * emb_dim, hidden_size) / ((block_size * emb_dim)**0.5)
b1 = torch.randn(hidden_size)
w2 = torch.randn(hidden_size, vocab_size) / (hidden_size ** 0.5)
b2 = torch.randn(vocab_size)

params = [C, w1, b1, w2, b2]
for p in params:
  p.requires_grad = True

In [58]:
import torch.nn.functional as F
ins = C[Xtr[20]].view(-1)
l1 = (ins @ w1 + b1).tanh()
out = F.softmax(l1 @ w2 + b2, dim=0)
print(out)

tensor([0.0004, 0.0114, 0.0107, 0.0050, 0.0093, 0.0024, 0.0151, 0.0041, 0.0077,
        0.0087, 0.0026, 0.0060, 0.0020, 0.0076, 0.0825, 0.0041, 0.0003, 0.0013,
        0.0009, 0.0128, 0.0152, 0.0008, 0.0181, 0.0225, 0.0248, 0.0025, 0.0270,
        0.0051, 0.2682, 0.0288, 0.0073, 0.0027, 0.0077, 0.0037, 0.0022, 0.0014,
        0.0013, 0.0026, 0.0025, 0.0060, 0.0162, 0.0054, 0.0047, 0.0011, 0.0205,
        0.0035, 0.0062, 0.0019, 0.0383, 0.0369, 0.0020, 0.0927, 0.0376, 0.0024,
        0.0072, 0.0056, 0.0007, 0.0004, 0.0007, 0.0183, 0.0209, 0.0086, 0.0095,
        0.0020, 0.0043, 0.0068], grad_fn=<SoftmaxBackward0>)


In [59]:
num_epochs = 1
batch_size = 16
train_len = len(Xtr) - 1
max_steps = (train_len // batch_size) * num_epochs
lr = 1

In [60]:
for step in range(max_steps):
  batch_indices = torch.randint(0, train_len, (batch_size,))
  x_batch = Xtr[batch_indices]
  y_batch = Ytr[batch_indices]
  emb = C[x_batch].view(batch_size, -1)
  l1 = (emb @ w1 + b1).tanh()

  out = F.softmax(l1 @ w2 + b2, dim=1)

  loss = -out[torch.arange(batch_size), y_batch].log().mean()


  for p in params:
    p.grad = None

  loss.backward()

  for p in params:
    p.data -= p.grad * lr

  if step % 10000 == 0:
    print("Step", str(step) + ", loss:", loss.item())
  elif step % 100 == 0:
    lr *= 0.9

Step 0, loss: 4.656942367553711
Step 10000, loss: 2.4907734394073486
Step 20000, loss: 1.818792700767517
Step 30000, loss: 1.6171382665634155
Step 40000, loss: 1.6939035654067993
Step 50000, loss: 1.475493311882019


In [76]:
def generate():
  out_str = ""
  context = [0] * block_size
  for x in range(500):
    c = torch.tensor(context)
    emb = C[c].view(-1)
    l1 = l1 = (emb @ w1 + b1).tanh()
    out = F.softmax(l1 @ w2 + b2, dim=0)
    index = torch.multinomial(out, num_samples=1)
    context = context[1:] + [index.item()]
    out_str += itos[index.item()]
  return out_str
print(generate())


SEMS:
Nat trourss,
Aspirese; brourannud, and robral!

IUOLIUES:
Whef ly Fith as alice
Tfu to thau and that, selint that eolest stip enetthin this youigt toke we istreat thILwer on arst thegpertitibs glock thk re hing one and tas thiaist in mastodr: I hur,
Oig on nos mak, sor!

Dock Vom Hin sicag fo fons then wiag beoti love? rol,
Sh
ES:
Whouls sudl soan dacatius fuptirizy'et Mingond ser to akave ase.
I wore make of herstre be groe melprine:

ell on moc sI
And lore, loond thou rt lowe:
Doue and 
